<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Twilio - Send Twilio SMS messages for Google Calendar Events
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/template.ipynb" target="_parent">
<img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/>
</a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=bug_report.md&title=[ERROR]+Tool+/+Folder+Action+of+the+notebook+">Bug report</a>

**Tags:** #googlecalendar #twilio #notification #event

**Author:** [Sriniketh Jayasendil](https://www.linkedin.com/in/sriniketh-jayasendil)

**Description:** This notebook is used to send notifications (SMS & Email) for new recurring events created using Google Calendar.

**References:**
- https://developers.google.com/calendar/api/quickstart/python

## Input

### Import libraries

In [1]:
import naas
try:
    from apiclient.discovery import build
    from google_auth_oauthlib.flow import InstalledAppFlow
except ModuleNotFoundError:
    !pip install google-api-python-client --user
    from apiclient.discovery import build
    from google_auth_oauthlib.flow import InstalledAppFlow 
try:
    from twilio.rest import Client
except ModuleNotFoundError:
    !pip install twilio --user
    from twilio.rest import Client
import pickle

### Setup Variables

1. [Twilio](https://console.twilio.com)

- Account SID
- Authentication Token
- My Twilio phone number
- Twilio verified phone number

2. [Google Cloud](https://console.cloud.google.com/)

- Follow this [blog](https://blog.sriniketh.design/send-sms-using-twilio-for-g-calendar-events-using-naas-template) to get the credentials from the Google Cloud Platform

Then you are all set to proceed further 🚀

In [2]:
# Inputs

## Twilio
account_sid = naas.secret.get("TWILIO_ACCOUNT_SID")
auth_token = naas.secret.get("TWILIO_AUTH_TOKEN")
twilio_phone_number = naas.secret.get("TWILIO_FROM_NUMBER")
twilio_verified_phone_number = naas.secret.get("TWILIO_TO_NUMBER")

## Google Calendar
scopes = ['https://www.googleapis.com/auth/calendar']
flow = InstalledAppFlow.from_client_secrets_file("./secrets.json", scopes=scopes)
credentials = flow.run_console()
service = build("calendar", "v3", credentials=credentials)

## New event details
event = {
  'summary': 'Naas.ai X Community call',
  'location': 'https://meet.google.com/czx-zjpb-fof',
  'description': 'Weekly community call to share the progress, roadmap, latest announcements, and lot more...',
  'start': {
    'dateTime': '2023-03-16T21:30:00+05:30',
    'timeZone': 'Asia/Kolkata',
  },
  'end': {
    'dateTime': '2023-03-16T22:30:00+05:30',
    'timeZone': 'Asia/Kolkata',
  },
  'recurrence': [
    'RRULE:FREQ=WEEKLY;BYDAY=TH'
  ],
  'attendees': [
    {'email': 'srinikethcr7@gmail.com'},
    {'email': 'd20l213@psgitech.ac.in'}
#     {'email': ''}
  ],
  'reminders': {
    'useDefault': False,
    'overrides': [
      {'method': 'email', 'minutes': 24 * 60},
      {'method': 'popup', 'minutes': 30},
    ],
  },
}

# Outputs
message = ''

FileNotFoundError: [Errno 2] No such file or directory: './secrets.json'

## Model

### Create new event with the data above

In [ ]:
event = service.events().insert(calendarId='primary', body=event).execute()
message = 'Event created and can be accessed at ' + event.get('htmlLink')
print('Event created📆')

### Send notifications of new calendar Events

In [ ]:
def send_sms_google_calendar_events(to_phone_number, message):
    # Send SMS using Twilio
    client = Client(account_sid, auth_token)
    sent_message = client.messages.create(
        from_=twilio_phone_number,
        to=twilio_verified_phone_number,
        body=message)
    

    print(sent_message)

## Output

### Save result in csv

In [ ]:
send_sms_google_calendar_events(to_phone_number=twilio_verified_phone_number, message=message)
print('\n\nDone🚀')